In [ ]:
# Imports
import streamlit as st
import uuid
from PIL import Image
import os
import hashlib

with st.spinner("Setting up the application... Please wait :)"):
    from models import visionmodels, textmodels

# Initialize models in session state
if "vision_model" not in st.session_state:
    st.session_state["vision_model"] = visionmodels.get_default_model()
if "text_model" not in st.session_state:
    st.session_state["text_model"] = textmodels.get_default_model()

VISION_MODEL = st.session_state["vision_model"]
TEXT_MODEL = st.session_state["text_model"]

def delete_image(name):
    ingredients_in_image = st.session_state['images'][name]['ingredients']
    for ingredient in ingredients_in_image:
        ingredient_count = st.session_state['ingredients'][ingredient]['count']
        if ingredient_count == 1:
            st.session_state['ingredients'].pop(ingredient)
        else:
            st.session_state['ingredients'][ingredient]['count'] -= 1
    st.session_state['images'].pop(name)
    st.session_state["file_uploader_key"] += 1
    st.rerun()

def setup_file_uploader():
    st.write('''<style>.uploadedFile {display: none;}</style>''', unsafe_allow_html=True)
    
    images_so_far = st.session_state['images']
    uploaded_files = st.file_uploader("Upload an image of your leftovers...", type=["jpg", "jpeg", "png"], accept_multiple_files=True, key=st.session_state["file_uploader_key"])
    
    if uploaded_files:
        for uploaded_file in uploaded_files:
            hash = hashlib.md5(uploaded_file.read()).hexdigest()
            if hash in images_so_far:
                print("Image already exists")
                continue
            else:
                image = Image.open(uploaded_file)
                image_path = os.path.join(os.getcwd(), str(uuid.uuid4()) + "." + uploaded_file.name.split(".")[-1])
                image.save(image_path)
                images_so_far[hash] = {"filepath": image_path, "image": image}
                print("Image saved")
    
    images_to_process = [image for image in images_so_far.keys() if "ingredients" not in images_so_far[image]]
    
    with st.spinner("Processing image(s)..."):
        if images_to_process:
            for image in images_to_process:
                image_path = images_so_far[image]["filepath"]
                ingredients = VISION_MODEL.run_model(image_path=image_path).items
                images_so_far[image]["ingredients"] = ingredients
                for ingredient in ingredients:
                    if ingredient not in st.session_state['ingredients']:
                        st.session_state['ingredients'][ingredient] = {'count': 1, 'selected': True}
                    else:
                        st.session_state['ingredients'][ingredient]['count'] += 1
                        st.session_state['ingredients'][ingredient]['selected'] = True
            st.rerun()
    
    st.write("## Uploaded Images")
    if images_so_far:
        for index, image in enumerate(images_so_far):
            columns = st.columns([5,1])
            with columns[0]:
                with st.expander(image):
                    st.image(images_so_far[image]['image'], use_column_width=True)
            with columns[1]:
                if st.button("Delete", key="delete_image" + str(index)):
                    delete_image(image)

def toggle_ingredient(ingredient):
    st.session_state['ingredients'][ingredient]['selected'] = not st.session_state['ingredients'][ingredient]['selected']

def ingredients_container(ingredients):
    st.header('Identified Ingredients')
    cols = st.columns(6)
    
    ingredients_list = list(ingredients.keys())
    for row_start_index in range(0, len(ingredients_list), 6):
        for col_index in range(6):
            if row_start_index + col_index < len(ingredients_list):
                ingredient = ingredients_list[row_start_index + col_index]
                selected = ingredients[ingredient]['selected']
                type = "primary" if selected else "secondary"
                cols[col_index].button(ingredient, key='dynamic_button_' + ingredient, type=type, on_click=toggle_ingredient, args=[ingredient], use_container_width=True)
    
    new_ingredient = st.text_input("Add a missing ingredient")
    if st.button('Add Ingredient'):
        if new_ingredient and new_ingredient not in ingredients:
            ingredients[new_ingredient] = {'count': 1, 'selected': True}
            st.rerun()

def setup_form():
    with st.form("recipe_form"):
        columns = st.columns(2)
        with columns[0]:
            diet = st.selectbox("Diet Preference", ("Vegetarian", "Non-Vegetarian", "Vegan"))
        with columns[1]:
            course = st.selectbox("Meal Type", ("Breakfast", "Lunch", "Dinner", "Snack"))
        columns = st.columns(2)
        with columns[0]:
            cuisine = st.text_input("Preferred Cuisine")
        with columns[1]:
            time_to_cook = st.slider("Max Cooking Time (mins)", 0, 120, 30)
        submitted = st.form_submit_button("Generate Recipe")
        
        if submitted:
            selected_ingredients = [ingredient for ingredient in st.session_state['ingredients'] if st.session_state['ingredients'][ingredient]['selected']]
            
            with st.spinner("Fetching recipe..."):
                data = {"cuisine": cuisine, "time": time_to_cook, "ingredients": selected_ingredients, "diet": diet, "course": course}
                recipe = TEXT_MODEL.run_model(data)
                
            st.write("## Your Recipe: " + recipe.name)
            
            with st.expander("### Ingredients"):
                for ingredient in recipe.ingredients:
                    st.write("- " + ingredient)
            
            with st.expander("### Steps"):
                for index, step in enumerate(recipe.steps):
                    st.write(f"{index + 1}. {step}")

def main():
    if "file_uploader_key" not in st.session_state:
        st.session_state["file_uploader_key"] = 0
    if 'ingredients' not in st.session_state:
        st.session_state['ingredients'] = {}
    if 'images' not in st.session_state:
        st.session_state['images'] = {}
    
    st.title("Dine Wise - Smart Recipes for Your Leftovers!")
    setup_file_uploader()
    ingredients_container(st.session_state["ingredients"])
    setup_form()

if __name__ == "__main__":
    main()
